# Tensorflow dataset 'CIFAR10' Load

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
train_datasets = tfds.load('cifar10', split='train')
valid_datasets = tfds.load('cifar10', split='test')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteBI0NRW/cifar10-train.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteBI0NRW/cifar10-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [3]:
train_datasets

<PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>

In [4]:
for data in train_datasets.take(5):
  image = tf.cast(data['image'],tf.float32) / 255.0
  label = data['label']
  print(image.shape)
  print(label)

(32, 32, 3)
tf.Tensor(7, shape=(), dtype=int64)
(32, 32, 3)
tf.Tensor(8, shape=(), dtype=int64)
(32, 32, 3)
tf.Tensor(4, shape=(), dtype=int64)
(32, 32, 3)
tf.Tensor(4, shape=(), dtype=int64)
(32, 32, 3)
tf.Tensor(6, shape=(), dtype=int64)


In [5]:
def preprocessing(data):
  image = tf.cast(data['image'], tf.float32) / 255.0
  label = data['label']
  return image, label

In [6]:
BATCH_SIZE = 128
train_data = train_datasets.map(preprocessing).shuffle(1000).batch(BATCH_SIZE)
valid_data = valid_datasets.map(preprocessing).batch(BATCH_SIZE)

In [7]:
for image, label in train_data.take(1):
  print(image.shape)
  print(label.shape)

(128, 32, 32, 3)
(128,)


# Sequential API 모델링

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [9]:
model = Sequential([
                    Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
                    MaxPooling2D(2, 2),
                    Conv2D(64, 3, activation='relu'),
                    MaxPooling2D(2, 2),
                    Flatten(),
                    Dense(32, activation='relu'),
                    Dense(10, activation='softmax'),
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                73760     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [12]:
model.fit(train_data,
          validation_data=(valid_data),
          epochs=10,
          )

Epoch 1/10
391/391 [==============================] - 45s 41ms/step - loss: 1.6692 - acc: 0.3989 - val_loss: 1.4528 - val_acc: 0.4925
Epoch 2/10
391/391 [==============================] - 9s 22ms/step - loss: 1.3119 - acc: 0.5354 - val_loss: 1.2157 - val_acc: 0.5784
Epoch 3/10
391/391 [==============================] - 8s 21ms/step - loss: 1.1647 - acc: 0.5929 - val_loss: 1.1310 - val_acc: 0.6073
Epoch 4/10
391/391 [==============================] - 8s 21ms/step - loss: 1.0746 - acc: 0.6259 - val_loss: 1.0728 - val_acc: 0.6337
Epoch 5/10
391/391 [==============================] - 8s 21ms/step - loss: 1.0107 - acc: 0.6491 - val_loss: 1.0096 - val_acc: 0.6554
Epoch 6/10
391/391 [==============================] - 8s 21ms/step - loss: 0.9632 - acc: 0.6672 - val_loss: 0.9742 - val_acc: 0.6711
Epoch 7/10
391/391 [==============================] - 9s 22ms/step - loss: 0.9196 - acc: 0.6840 - val_loss: 0.9597 - val_acc: 0.6706
Epoch 8/10
391/391 [==============================] - 9s 22ms/step -

# Functional API 모델링

In [13]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [20]:
input_ = Input(shape=((32, 32, 3)))

In [28]:
x = Conv2D(32, 3, activation='relu')(input_)
x = MaxPooling2D(2, 2)(x)
x = Conv2D(64, 3, activation='relu')(x)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

In [29]:
model = Model(input_, x)

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                73760 

In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [32]:
model.fit(train_data,
          validation_data=(valid_data),
          epochs=10,
          )

Epoch 1/10
391/391 [==============================] - 9s 22ms/step - loss: 1.6659 - acc: 0.3905 - val_loss: 1.4003 - val_acc: 0.4924
Epoch 2/10
391/391 [==============================] - 8s 21ms/step - loss: 1.3320 - acc: 0.5244 - val_loss: 1.2593 - val_acc: 0.5464
Epoch 3/10
391/391 [==============================] - 8s 21ms/step - loss: 1.2020 - acc: 0.5758 - val_loss: 1.1688 - val_acc: 0.5831
Epoch 4/10
391/391 [==============================] - 8s 21ms/step - loss: 1.1136 - acc: 0.6079 - val_loss: 1.1025 - val_acc: 0.6134
Epoch 5/10
391/391 [==============================] - 8s 21ms/step - loss: 1.0457 - acc: 0.6341 - val_loss: 1.0335 - val_acc: 0.6440
Epoch 6/10
391/391 [==============================] - 8s 21ms/step - loss: 0.9858 - acc: 0.6562 - val_loss: 1.0063 - val_acc: 0.6490
Epoch 7/10
391/391 [==============================] - 8s 21ms/step - loss: 0.9423 - acc: 0.6722 - val_loss: 0.9920 - val_acc: 0.6595
Epoch 8/10
391/391 [==============================] - 8s 21ms/step - 

# train_on_batch 커스텀 학습

batch 단위의 학습을 한 다음에 batch 별로 loss를 확인하거나 시각화

고급학습, 특히 GAN 같은 곳에서 활용


In [33]:
model = Sequential([
                    Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
                    MaxPooling2D(2, 2),
                    Conv2D(64, 3, activation='relu'),
                    MaxPooling2D(2, 2),
                    Flatten(),
                    Dense(32, activation='relu'),
                    Dense(10, activation='softmax'),
])

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                73760     
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [35]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [36]:
EPOCHS = 10

for epoch in range(EPOCHS):
  for batch, (image, label) in train_data.enumerate():
    loss = model.train_on_batch(image, label)
    print(f'epoch: {epoch + 1}, batch: {batch + 1}, loss: {loss[0]:.3f}, acc: {loss[1]:.2f}')

epoch: 1, batch: 1, loss: 2.312, acc: 0.09
epoch: 1, batch: 2, loss: 2.294, acc: 0.18
epoch: 1, batch: 3, loss: 2.280, acc: 0.10
epoch: 1, batch: 4, loss: 2.276, acc: 0.14
epoch: 1, batch: 5, loss: 2.304, acc: 0.12
epoch: 1, batch: 6, loss: 2.262, acc: 0.15
epoch: 1, batch: 7, loss: 2.268, acc: 0.15
epoch: 1, batch: 8, loss: 2.289, acc: 0.12
epoch: 1, batch: 9, loss: 2.283, acc: 0.13
epoch: 1, batch: 10, loss: 2.283, acc: 0.16
epoch: 1, batch: 11, loss: 2.258, acc: 0.19
epoch: 1, batch: 12, loss: 2.249, acc: 0.21
epoch: 1, batch: 13, loss: 2.275, acc: 0.14
epoch: 1, batch: 14, loss: 2.242, acc: 0.13
epoch: 1, batch: 15, loss: 2.279, acc: 0.10
epoch: 1, batch: 16, loss: 2.258, acc: 0.14
epoch: 1, batch: 17, loss: 2.209, acc: 0.26
epoch: 1, batch: 18, loss: 2.221, acc: 0.15
epoch: 1, batch: 19, loss: 2.245, acc: 0.15
epoch: 1, batch: 20, loss: 2.210, acc: 0.18
epoch: 1, batch: 21, loss: 2.202, acc: 0.16
epoch: 1, batch: 22, loss: 2.174, acc: 0.18
epoch: 1, batch: 23, loss: 2.172, acc: 0.

# GradientTape 커스텀 학습

In [37]:
model = Sequential([
                    Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
                    MaxPooling2D(2, 2),
                    Conv2D(64, 3, activation='relu'),
                    MaxPooling2D(2, 2),
                    Flatten(),
                    Dense(32, activation='relu'),
                    Dense(10, activation='softmax'),
])

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                73760     
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [40]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [41]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_acc')

In [49]:
@tf.function

def train_step(image, label):
  with tf.GradientTape() as tape:
    prediction = model(image, training=True)
    loss = loss_function(label, prediction)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(label, prediction)

In [50]:
@tf.function

def valid_step(image, label):
  prediction = model(image, training=False)
  loss = loss_function(label, prediction)

  valid_loss(loss)
  valid_acc(label, prediction)

In [51]:
EPOCHS = 10

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_acc.reset_states()
  valid_loss.reset_states()
  valid_acc.reset_states()

  for image, label in train_data:
    train_step(image, label)

  for image, label in valid_data:
    valid_step(image, label)

  print(f'epoch :{epoch+1}, loss :{train_loss.result()}, acc: {train_acc.result()}, val_loss: {valid_loss.result()}, val_accc: {valid_acc.result()}')

epoch :1, loss :1.6576952934265137, acc: 0.39976000785827637, val_loss: 1.4008327722549438, val_accc: 0.501800000667572
epoch :2, loss :1.3097424507141113, acc: 0.5357000231742859, val_loss: 1.2179440259933472, val_accc: 0.5687999725341797
epoch :3, loss :1.1825929880142212, acc: 0.5863199830055237, val_loss: 1.154996633529663, val_accc: 0.593500018119812
epoch :4, loss :1.0932848453521729, acc: 0.6168799996376038, val_loss: 1.093947410583496, val_accc: 0.6190000176429749
epoch :5, loss :1.031445860862732, acc: 0.6390399932861328, val_loss: 1.055542230606079, val_accc: 0.6377999782562256
epoch :6, loss :0.9823026061058044, acc: 0.6575199961662292, val_loss: 1.0182693004608154, val_accc: 0.6517999768257141
epoch :7, loss :0.9410445094108582, acc: 0.6736400127410889, val_loss: 0.9803670644760132, val_accc: 0.6664000153541565
epoch :8, loss :0.9064726829528809, acc: 0.6856399774551392, val_loss: 0.9564294815063477, val_accc: 0.6743999719619751
epoch :9, loss :0.8765926957130432, acc: 0.69